# Pure Python

In [1]:
def pure_python_simple(filename, maketuple=False):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    if maketuple:
        full_dict = tuple(full_dict.values())
    with open(filename, 'rb') as f:
        print(sum(full_dict[i] for i in f.read()))

In [2]:
%%timeit -n 1 -r 1
pure_python_simple("random-20221005.txt")

9140634224
7.29 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
%%timeit -n 1 -r 1
pure_python_simple("random-20221005.txt", maketuple=True)

9140634224
7.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Numpy

In [4]:
import numpy as np
def numpy_indexing(filename, do_loops=False):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)
    with open(filename, 'rb') as f:
        if do_loops:
            total = 0
            temp = 1
            loops = 0
            while temp:
                temp = np.sum(full_dict[np.frombuffer(f.read1(100000), dtype=np.uint8)])
                total += temp
                loops += 1
            print(total)
            print(loops)
        else:
            print(np.sum(full_dict[np.frombuffer(f.read(), dtype=np.uint8)]))

In [5]:
#Error
np.sum(np.array([800]*10000000))

-589934592

In [6]:
%%timeit -n 1 -r 1
numpy_indexing("random-20221005.txt")

550699632
617 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%%timeit
numpy_indexing("random-20221005.txt", do_loops=True)

9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
345 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Pure Numba

In [8]:
from numba import jit
@jit(nogil=True, fastmath=True)
def gematria_sum(bytearr):
    total = 0
    for char in bytearr:
        if char < 65:
            continue
        if char > 96:
            char -= 32
        if char < 74: #A 4- J
            total += char-64
            continue
        if char < 84: #K - S
            total += (char-73)*10
            continue
        if char < 91: #T - Z
            total += (char-82)*100
    return total
gematria_sum(b'qwertyuiopasdfghjklzxcvbnm') #to compile

4095

In [9]:
def numba_basic(filename, do_loops=False):
    with open(filename, 'rb') as f:
        if do_loops:
            total = 0
            temp = 1
            loops = 0
            while temp:
                temp = gematria_sum(f.read1(100000))
                total += temp
                loops += 1
            print(total)
            print(loops)
        else:
            print(gematria_sum(f.read()))

In [10]:
%%timeit -n 1 -r 1
numba_basic("random-20221005.txt")

9140634224
784 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
%%timeit -n 1
numba_basic("random-20221005.txt", do_loops=True)

9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
9140634224
1050
762 ms ± 30.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# multithread numpy

In [12]:
import threading

def threaded_numpy(filename):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint32)
    
    lock = threading.Lock()
    results = []
    def do_job(full_dict, bytearr, results, lock):
        num = np.sum(full_dict[np.frombuffer(bytearr, dtype=np.uint8)])
        with lock:
            results.append(num)
    threads = []
    
    with open(filename, 'rb') as f:
        total = 0
        loops = 0
        temp = f.read1(1000000)
        while len(temp):
            threads.append(threading.Thread(target=do_job, 
                                            args=(full_dict, temp, results, lock)))
            threads[-1].start()
            loops += 1
            temp = f.read1(1000000)
            if len(threads)>10:
                threads[0].join()
                del threads[0]
        for thread in threads:
            thread.join()
        print(sum(results))
        print(loops)

In [13]:
%%timeit -n 1
threaded_numpy("random-20221005.txt")

9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
9140634224
105
197 ms ± 9.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
import numpy as np
import os
from concurrent.futures import ThreadPoolExecutor

def multithread_numpy_fast_gematria(filename, chunksize=400000, max_threads=12):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.uint16)
    task = lambda full_dict, bytearr: np.sum(full_dict[np.frombuffer(bytearr, dtype=np.uint8)])
    threads = []
    with open(filename, 'rb', chunksize) as f:
        filesize = os.fstat(f.fileno()).st_size
        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            for _ in range(1 + filesize//chunksize):
                threads.append(executor.submit(task, full_dict, f.read(chunksize)))
    return sum(thread.result() for thread in threads)

In [15]:
print(multithread_numpy_fast_gematria("E:/random-20221005.txt"))

9140634224


In [16]:
%%timeit -n 1 -r 10
multithread_numpy_fast_gematria("E:/random-20221005.txt")

79.2 ms ± 7.25 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


# Numba, as fast as possible

In [17]:
import numpy as np
from numba import jit
import os
from concurrent.futures import ThreadPoolExecutor

@jit(nogil=True, fastmath=True)
def counter(full_dict, bytearr):
    total = 0
    for i in bytearr:
        total += full_dict[i]
    return total
_ = counter(np.zeros((128,), dtype=np.int32), b'abc') #to compile

def final_fast_gematria(filename, chunksize=600000, max_threads=4):
    maindict = {65: 1, 66: 2, 67: 3, 68: 4, 69: 5, 70: 6, 71: 7, 72: 8, 73: 9, 74: 10, \
                75: 20, 76: 30, 77: 40, 78: 50, 79: 60, 80: 70, 81: 80, 82: 90, 83: 100, \
                84: 200, 85: 300, 86: 400, 87: 500, 88: 600, 89: 700, 90: 800}
    full_dict = {**{i:0 for i in range(128)}, **maindict, **{key+32:val for key,val in maindict.items()}}
    full_dict = np.array(tuple(full_dict.values()), dtype=np.int32)    
    threads = []
    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        with open(filename, 'rb', 0) as f:
            filesize = os.fstat(f.fileno()).st_size
            for _ in range(filesize//chunksize - 1):
                threads.append(executor.submit(counter, full_dict, f.read(chunksize)))
            return counter(full_dict, f.read()) + sum(thread.result() for thread in threads) 

In [18]:
print(final_fast_gematria("E:/random-20221005.txt"))

9140634224


In [19]:
%%timeit -n 2 -r 10
final_fast_gematria("E:/random-20221005.txt")

27.9 ms ± 1.11 ms per loop (mean ± std. dev. of 10 runs, 2 loops each)


In [20]:
def only_read(filename, chunksize=600000):
    with open(filename, 'rb', chunksize) as f:
        filesize = os.fstat(f.fileno()).st_size
        for _ in range(1 + filesize//chunksize):
            f.read(600000)

In [21]:
%%timeit -n 1 -r 10
only_read("E:/random-20221005.txt")

22.6 ms ± 985 µs per loop (mean ± std. dev. of 10 runs, 1 loop each)
